In [ ]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
import numpy as np
from tensorflow.keras.regularizers import l2
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.models import load_model

In [ ]:
# Загрузка данных
df = pd.read_csv("Dataset.csv") 

In [ ]:
# Варианты выбора наборов данных
#features = ['ONI', 'Nino3.4_SST', 'South_Sea_SST_t+3']
features = ['ONI', 'Nino3.4_SST']
#features = ['ONI', 'South_Sea_SST_t+3']
#features = ['ONI']

# Горизонт предсказания
horizon = 3

# Шширина окна
W = 24

# Параметры модели
lstm_units=64 # количество нейроной в LSTM слое
dense_units=32 # количество нейроной в обычном слое
dropout_rate=0.2 # доля нейронов, которую модель забывает

# Параметры обучения
epochs=100 # количество эпох
batch_size=32 # размер батча
patience=5 # параматр EarlyStopping (сколько эпох без повышения точности ждать перед остановкой)

In [ ]:
df = df[['Date']+ features]

In [ ]:
# Разбиение на обучающий, валидационный и тестовый наборы
train_df = df.iloc[:744]
val_df   = df.iloc[744:828]
test_df  = df.iloc[828:]

In [ ]:
# Инициализация стандартизатора для признаков
scaler = StandardScaler()
# Обучение стандаритизатора по обучающей выборке
scaler.fit(train_df[features])

# Стандартизация каждого набора
train_values = scaler.transform(train_df[features])
val_values   = scaler.transform(val_df[features])
test_values  = scaler.transform(test_df[features])

In [ ]:
# Функция для формирования выборки (обучающей / валидационной / тестовой)
def create_sequences(data_array, window_size, horizon):
    X_seq, y_seq = [], []
    N = len(data_array)
    oni_index = 0  
    for start in range(0, N - window_size - horizon + 1):
        end = start + window_size
        seq_x = data_array[start : end]            
        target_y = data_array[end + horizon - 1, oni_index]
        X_seq.append(seq_x)
        y_seq.append(target_y)
    X_seq = np.array(X_seq)
    y_seq = np.array(y_seq)
    return X_seq, y_seq

In [ ]:
# Функция для инициализации модели
def create_model(window_size, lstm_units, dense_units,  dropout_rate):
    model = Sequential([
        Input(shape=(window_size, len(features))),
        LSTM(lstm_units, activation='tanh'),
        Dropout(dropout_rate),
        Dense(dense_units, activation='relu', kernel_regularizer=l2(0.001)),
        Dense(1)
    ])
    optimizer = Adam(learning_rate=0.0005)
    model.compile(optimizer=optimizer, loss='mse', metrics=['mae'])

    return model

In [ ]:
# Колбэк для ранней остановки
es = EarlyStopping(monitor='val_loss', patience=patience, restore_best_weights=True)

In [ ]:
num_features = len(features)

# Формирование обучающей, валидационно. и тестовой выборок для заданного горизонта
X_train, y_train = create_sequences(train_values, W, horizon)
X_val,   y_val   = create_sequences(val_values,   W, horizon)
X_test,  y_test  = create_sequences(test_values,  W, horizon)


In [ ]:
# Инициализация модели
model = create_model(window_size=W, lstm_units=lstm_units, dense_units=dense_units, dropout_rate=dropout_rate)

# Обучение модели
model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size,
                    validation_data=(X_val, y_val), 
                    callbacks=[es], verbose=0)

In [ ]:
# Предсказание
y_pred = model.predict(X_test)

# Обратное преобразование данных
y_test_inv = scaler.inverse_transform(
    np.concatenate([y_test.reshape(-1,1), 
                    np.zeros((len(y_test), len(features)-1))], axis=1)
)[:, 0]
y_pred_inv = scaler.inverse_transform(
    np.concatenate([y_pred, 
                    np.zeros((len(y_pred), len(features)-1))], axis=1)
)[:, 0]

In [ ]:
# Сохранение модели с горизонтом предсказания horizon
#model.save(f"LSTM_with_horizon_{horizon}.h5")

# Загрузка модели с горизонтом предсказания horizon
#model = load_model(f"LSTM_with_horizon_{horizon}.h5")